In [ ]:
from ortools.linear_solver import pywraplp

# Parameters
demand = [20, 30, 40, 35, 25]  # Forecasted daily demand
holding_cost = 2  # Cost per unit of inventory held
order_cost = 50  # Fixed order cost
capacity = 100  # Maximum inventory capacity
# Solver
solver = pywraplp.Solver.CreateSolver('SCIP')
# Variables
order = [solver.BoolVar(f'order_{t}') for t in range(len(demand))]
inventory = [solver.IntVar(0, capacity, f'inventory_{t}') for t in range(len(demand))]
# Constraints
solver.Add(inventory[0] == 0)  # Starting inventory is zero
for t in range(1, len(demand)):
    solver.Add(inventory[t] == inventory[t-1] + 100 * order[t] - demand[t])
# Objective: Minimize total cost
total_cost = sum(order_cost * order[t] + holding_cost * inventory[t] for t in range(len(demand)))
solver.Minimize(total_cost)
# Solve
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    for t in range(len(demand)):
        print(f"Day {t+1}: Order = {order[t].solution_value()}, Inventory = {inventory[t].solution_value()}")
else:
    print("No optimal solution found.")

ModuleNotFoundError: No module named 'ortools'

In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import numpy as np
import pandas as pd

# Generate synthetic time series data
np.random.seed(42)
n_samples = 100
n_timestamps = 50
# Create time series with two classes
class_0 = np.random.normal(0, 1, (n_samples // 2, n_timestamps))
class_1 = np.random.normal(2, 1, (n_samples // 2, n_timestamps))
# Combine data and labels
X = np.vstack((class_0, class_1))
y = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
# Convert to DataFrame
df = pd.DataFrame(X)
df['label'] = y
print(df.head())


from transformers import AutoTokenizer

import os
os.environ["WANDB_DISABLED"] = "true"


# Use a tokenizer to prepare data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_time_series(series):
    series_str = " ".join(map(str, series))
    return tokenizer(series_str, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
# Tokenize the dataset
tokens = [tokenize_time_series(row[:-1].values) for _, row in df.iterrows()]
labels = df['label'].values

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Split data into train and test sets
train_tokens, test_tokens, train_labels, test_labels = train_test_split(tokens, labels, test_size=0.2, random_state=42)
# Create dataset objects
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {key: val.squeeze() for key, val in self.tokens[idx].items()}, self.labels[idx]
train_dataset = TimeSeriesDataset(train_tokens, train_labels)
test_dataset = TimeSeriesDataset(test_tokens, test_labels)
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# Set up Trainer
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# Fine-tune the model
trainer.train()

from sklearn.metrics import accuracy_score

# Make predictions
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Test Accuracy: {accuracy:.2f}")

          0         1         2         3         4         5         6  \
0  0.496714 -0.138264  0.647689  1.523030 -0.234153 -0.234137  1.579213   
1  0.324084 -0.385082 -0.676922  0.611676  1.031000  0.931280 -0.839218   
2 -1.415371 -0.420645 -0.342715 -0.802277 -0.161286  0.404051  1.886186   
3  0.250493  0.346448 -0.680025  0.232254  0.293072 -0.714351  1.865775   
4  0.357787  0.560785  1.083051  1.053802 -1.377669 -0.937825  0.515035   

          7         8         9  ...        41        42        43        44  \
0  0.767435 -0.469474  0.542560  ...  0.171368 -0.115648 -0.301104 -1.478522   
1 -0.309212  0.331263  0.975545  ...  0.968645 -0.702053 -0.327662 -0.392108   
2  0.174578  0.257550 -0.074446  ...  1.307143 -1.607483  0.184634  0.259883   
3  0.473833 -1.191303  0.656554  ...  0.856399  0.214094 -1.245739  0.173181   
4  0.513786  0.515048  3.852731  ... -0.114736  0.504987  0.865755 -1.200296   

         45        46        47        48        49  label  
0 -0.71

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import (
    AutoTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)

# 1. Generate synthetic time series data
np.random.seed(42)
n_samples = 100
n_timestamps = 50

class_0 = np.random.normal(0, 1, (n_samples // 2, n_timestamps))
class_1 = np.random.normal(2, 1, (n_samples // 2, n_timestamps))
X = np.vstack((class_0, class_1))
y = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))

df = pd.DataFrame(X)
df['label'] = y
print(df.head())

# 2. Tokenize the time series data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_time_series(series):
    series_str = " ".join(map(str, series))
    return tokenizer(series_str, truncation=True, padding="max_length", max_length=128, return_tensors="pt")

tokens = [tokenize_time_series(row[:-1].values) for _, row in df.iterrows()]
labels = df['label'].values

# 3. Prepare dataset for Trainer
train_tokens, test_tokens, train_labels, test_labels = train_test_split(tokens, labels, test_size=0.2, random_state=42)

class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val.squeeze(0) for key, val in self.tokens[idx].items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = TimeSeriesDataset(train_tokens, train_labels)
test_dataset = TimeSeriesDataset(test_tokens, test_labels)

# 4. Load model and training setup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"  # disables wandb and other loggers
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 5. Train and evaluate
trainer.train()

predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Test Accuracy: {accuracy:.2f}")


          0         1         2         3         4         5         6  \
0  0.496714 -0.138264  0.647689  1.523030 -0.234153 -0.234137  1.579213   
1  0.324084 -0.385082 -0.676922  0.611676  1.031000  0.931280 -0.839218   
2 -1.415371 -0.420645 -0.342715 -0.802277 -0.161286  0.404051  1.886186   
3  0.250493  0.346448 -0.680025  0.232254  0.293072 -0.714351  1.865775   
4  0.357787  0.560785  1.083051  1.053802 -1.377669 -0.937825  0.515035   

          7         8         9  ...        41        42        43        44  \
0  0.767435 -0.469474  0.542560  ...  0.171368 -0.115648 -0.301104 -1.478522   
1 -0.309212  0.331263  0.975545  ...  0.968645 -0.702053 -0.327662 -0.392108   
2  0.174578  0.257550 -0.074446  ...  1.307143 -1.607483  0.184634  0.259883   
3  0.473833 -1.191303  0.656554  ...  0.856399  0.214094 -1.245739  0.173181   
4  0.513786  0.515048  3.852731  ... -0.114736  0.504987  0.865755 -1.200296   

         45        46        47        48        49  label  
0 -0.71

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.419000


Test Accuracy: 1.00
